In [1]:
!unzip /content/data.zip -d /content/

Archive:  /content/data.zip
   creating: /content/data/
  inflating: /content/data/annotation.conf  
   creating: /content/data/dev/
  inflating: /content/data/dev/119336_text.ann  
  inflating: /content/data/dev/119336_text.txt  
  inflating: /content/data/dev/120043_text.ann  
  inflating: /content/data/dev/120043_text.txt  
  inflating: /content/data/dev/120183_text.ann  
  inflating: /content/data/dev/120183_text.txt  
  inflating: /content/data/dev/120890_text.ann  
  inflating: /content/data/dev/120890_text.txt  
  inflating: /content/data/dev/121809_text.ann  
  inflating: /content/data/dev/121809_text.txt  
  inflating: /content/data/dev/1277254_text.ann  
  inflating: /content/data/dev/1277254_text.txt  
  inflating: /content/data/dev/1530089_text.ann  
  inflating: /content/data/dev/1530089_text.txt  
  inflating: /content/data/dev/1535267_text.ann  
  inflating: /content/data/dev/1535267_text.txt  
  inflating: /content/data/dev/1551405_text.ann  
  inflating: /content/data/

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
import os
import numpy as np

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
from datasets import Dataset
from typing import List, Dict

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cpu


In [5]:
ENTITIES = ['CITY', 'COUNTRY', 'PERSON', 'ORGANIZATION']
TRAIN_PATH = './data/train/'
TEST_PATH = './data/test/'
DEV_PATH = './data/dev/'

In [6]:
path = os.path.join('.','data')

train_text, train_ann = [], []
test_text, test_ann = [], []
dev_text, dev_ann = [], []

if os.path.exists(path):
    print('Path is exist')
else:
    print('Path isn\'t exist')

for root, dirs, files in os.walk(path):
    for file in files:
        if 'train' in root:
            if file.endswith('.txt'):
                train_text.append(file)
            elif file.endswith('.ann'):
                train_ann.append(file)
        elif 'test' in root:
            if file.endswith('.txt'):
                test_text.append(file)
            elif file.endswith('.ann'):
                test_ann.append(file)
        elif 'dev' in root:
            if file.endswith('.txt'):
                dev_text.append(file)
            elif file.endswith('.ann'):
                dev_ann.append(file)

Path is exist


In [7]:
train_text.sort()
train_ann.sort()

test_text.sort()
test_ann.sort()

dev_text.sort()
dev_ann.sort()

In [8]:
print(dev_text[0], dev_ann[0])

119336_text.txt 119336_text.ann


In [9]:
print(f'Shape of train: {len(train_text)}')
print(f'Shape of test: {len(test_text)}')
print(f'Shape of dev: {len(dev_text)}')

Shape of train: 746
Shape of test: 93
Shape of dev: 94


In [10]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def read_txt_file(file_path: str):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def read_ann_file(file_path: str):
    entities = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split('\t')
            if 'T' in parts[0] :
                if ';' in parts[1]:
                    for part_entity in parts[1].split(';'):
                        if len(part_entity.split()) == 3:
                            entity_type, start, end = part_entity.split()
                        else:
                            start, end = part_entity.split()
                else:
                    entity_type, start, end = parts[1].split()

                if entity_type in ENTITIES:
                    entities.append((start, end, entity_type))

    return entities

def create_dataset(path:str, txt_files: List[str], ann_files: List[str]):
    data = []

    for txt_file, ann_file in zip(txt_files, ann_files):
        text = read_txt_file(path + txt_file)
        entities = read_ann_file(path + ann_file)
        tokens = text.split()
        ner_tags = ['O'] * len(tokens)

        for entity in entities:
            start, end, entity_type = entity

            start_token_idx = None
            end_token_idx = None
            cur_pos = 0

            for i, token in enumerate(tokens):
                token_start = text.find(token, cur_pos)
                token_end = token_start + len(token)

                if token_start == int(start):
                    start_token_idx = i

                if token_end == int(end):
                    end_token_idx = i
                    break
                cur_pos = token_end + 1

            if start_token_idx is not None and end_token_idx is not None:
                if start_token_idx == end_token_idx:
                    ner_tags[start_token_idx] = f'U-{entity_type}'
                else:
                    ner_tags[start_token_idx] = f'B-{entity_type}'
                    for i in range(start_token_idx + 1, end_token_idx):
                        ner_tags[i] = f'I-{entity_type}'
                    ner_tags[end_token_idx] = f'L-{entity_type}'

        data.append({
            'tokens': tokens,
            'ner_tags': ner_tags
        })

    return data

In [12]:
train_data = create_dataset(TRAIN_PATH, train_text, train_ann)
test_data = create_dataset(TEST_PATH, test_text, test_ann)
dev_data = create_dataset(DEV_PATH, dev_text, dev_ann)

In [13]:
def ner_tags_to_int(data: Dict[Dict,Dict], int_tags: Dict[str, int], max_int_tag: int):
    all_new_ner_tags = []

    for d in data:
        new_ner_tags = []
        for tag in d['ner_tags']:
            if tag not in int_tags.values():
                int_tags[max_int_tag] = tag
                max_int_tag += 1

            new_ner_tags.append(max_int_tag)

        all_new_ner_tags.append(new_ner_tags)

    return (all_new_ner_tags, int_tags, max_int_tag)

In [14]:
int_tags = {}
max_int_tag = 0

all_new_ner_tags, int_tags, max_int_tag = ner_tags_to_int(train_data, int_tags, max_int_tag)
train_data = Dataset.from_dict({'tokens': [d['tokens'] for d in train_data], 'ner_tags': all_new_ner_tags})

all_new_ner_tags, int_tags, max_int_tag = ner_tags_to_int(test_data, int_tags, max_int_tag)
test_data = Dataset.from_dict({'tokens': [d['tokens'] for d in test_data], 'ner_tags': all_new_ner_tags})

all_new_ner_tags, int_tags, max_int_tag = ner_tags_to_int(dev_data, int_tags, max_int_tag)
dev_data = Dataset.from_dict({'tokens': [d['tokens'] for d in dev_data], 'ner_tags': all_new_ner_tags})

In [15]:
max_int_tag

17

In [16]:
int_tags

{0: 'O',
 1: 'U-CITY',
 2: 'B-ORGANIZATION',
 3: 'I-ORGANIZATION',
 4: 'U-COUNTRY',
 5: 'U-ORGANIZATION',
 6: 'L-ORGANIZATION',
 7: 'B-CITY',
 8: 'L-CITY',
 9: 'B-PERSON',
 10: 'L-PERSON',
 11: 'U-PERSON',
 12: 'B-COUNTRY',
 13: 'L-COUNTRY',
 14: 'I-COUNTRY',
 15: 'I-PERSON',
 16: 'I-CITY'}

In [17]:
def tokenize_and_align_labels(data: Dataset):
    tokenized_inputs = tokenizer(data['tokens'], is_split_into_words=True, truncation=True, padding=True)

    labels = []

    for i, label in enumerate(data['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index = i)
        prev_word_idx = None
        label_ids =[]

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)

            prev_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels

    return tokenized_inputs

In [18]:
train_data = train_data.map(tokenize_and_align_labels, batched=True)
test_data = test_data.map(tokenize_and_align_labels, batched=True)
dev_data = dev_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/746 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

In [23]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=max_int_tag)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    logging_steps=1,
    report_to="none"  # Отключение wandb
)

trainer = Trainer(
    model= model,
    args= training_args,
    train_dataset= train_data,
    eval_dataset= dev_data
)

In [21]:
trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

In [ ]:
int_tags

{0: 'O',
 1: 'U-CITY',
 2: 'B-ORGANIZATION',
 3: 'I-ORGANIZATION',
 4: 'U-COUNTRY',
 5: 'U-ORGANIZATION',
 6: 'L-ORGANIZATION',
 7: 'B-CITY',
 8: 'L-CITY',
 9: 'B-PERSON',
 10: 'L-PERSON',
 11: 'U-PERSON',
 12: 'B-COUNTRY',
 13: 'L-COUNTRY',
 14: 'I-COUNTRY',
 15: 'I-PERSON',
 16: 'I-CITY'}

In [129]:
def get_entities(data: List[Dict], length_ann_file: int, int_tags: Dict[str, int]):
    words_entities = []
    for d in data:
        entity = d['entity']
        word = d['word']
        label = int(entity.split('_')[1])
        if label != 0:
            ner_tag = int_tags[label]
            words_entities.append((word, ner_tag))

    inferences = [None] * length_ann_file
    current_entity = None
    current_type = None

    for i, (word, entity) in enumerate(words_entities):
        mark, entity_type = entity.split('-')

        if mark == 'U':
            inferences[i] = (word, entity_type)
        elif mark == 'B':
            if current_entity is not None:
                inferences[i - 1] = (current_entity, current_type)
            current_entity = word
            current_type = entity_type
        elif mark == 'I':
            if current_entity is not None:
                current_entity += ' ' + word
        elif mark == 'L':
            if current_entity is not None:
                current_entity += ' ' + word
                inferences[i] = (current_entity, current_type)
                current_entity = None
                current_type = None

    inferences = [inf for inf in inferences if inf is not None]

    return inferences



In [99]:
text = read_txt_file(TEST_PATH + test_text[1])

In [100]:
text

'Глава департамента ЦБ РФ Надежда Иванова получила статус зампреда\n\nИванова, которой 13 июня исполнилось 60 лет, всю свою жизнь проработала в системе ЦБ. Сводный экономический департамент Банка России возглавляет с 1995 года.\nЗдание Центрального банка РФ. Архив\n\nДиректор сводного экономического департамента Банка России Надежда Иванова назначена также на должность заместителя председателя ЦБ, сообщил в четверг регулятор.\n\n\n\nИванова, у которой 13 июня был 60-летний юбилей, работает в системе Банка России (ранее — Госбанка СССР) с окончания института, то есть с 1975 года. Сводный экономический департамент возглавляет почти 20 лет — с 1995 года.\n\nИванова входит в совет директоров Центробанка. До сводного экономического департамента она трудилась в департаменте банковского надзора.\n\nСводный экономический департамент входит в блок денежно-кредитной политики.\n\nЭто первое назначение нового председателя Банка России Эльвиры Набиуллиной на этом посту. Раньше в руководстве Центроб

In [104]:
ann_f = read_ann_file(TEST_PATH + test_ann[1])

In [101]:
read_ann_file(TEST_PATH + test_ann[1])

[('19', '21', 'ORGANIZATION'),
 ('25', '40', 'PERSON'),
 ('149', '151', 'ORGANIZATION'),
 ('153', '199', 'ORGANIZATION'),
 ('232', '253', 'ORGANIZATION'),
 ('308', '320', 'ORGANIZATION'),
 ('321', '336', 'PERSON'),
 ('427', '434', 'PERSON'),
 ('495', '507', 'ORGANIZATION'),
 ('517', '530', 'ORGANIZATION'),
 ('576', '609', 'ORGANIZATION'),
 ('651', '658', 'PERSON'),
 ('668', '696', 'ORGANIZATION'),
 ('701', '737', 'ORGANIZATION'),
 ('754', '786', 'ORGANIZATION'),
 ('789', '822', 'ORGANIZATION'),
 ('921', '940', 'PERSON'),
 ('977', '988', 'ORGANIZATION'),
 ('1102', '1116', 'PERSON'),
 ('1132', '1167', 'ORGANIZATION'),
 ('1193', '1208', 'PERSON'),
 ('1251', '1268', 'ORGANIZATION'),
 ('22', '24', 'COUNTRY'),
 ('19', '24', 'ORGANIZATION'),
 ('187', '199', 'ORGANIZATION'),
 ('193', '199', 'COUNTRY'),
 ('251', '253', 'COUNTRY'),
 ('271', '320', 'ORGANIZATION'),
 ('314', '320', 'COUNTRY'),
 ('391', '393', 'ORGANIZATION'),
 ('685', '696', 'ORGANIZATION'),
 ('908', '920', 'ORGANIZATION'),
 ('106

In [127]:
ner_pipeline(text)

[{'entity': 'LABEL_0',
  'score': 0.8255442,
  'index': 1,
  'word': 'Глава',
  'start': 0,
  'end': 5},
 {'entity': 'LABEL_2',
  'score': 0.49343318,
  'index': 2,
  'word': 'департамента',
  'start': 6,
  'end': 18},
 {'entity': 'LABEL_2',
  'score': 0.3586389,
  'index': 3,
  'word': 'ЦБ',
  'start': 19,
  'end': 21},
 {'entity': 'LABEL_4',
  'score': 0.3692087,
  'index': 4,
  'word': 'РФ',
  'start': 22,
  'end': 24},
 {'entity': 'LABEL_9',
  'score': 0.8299378,
  'index': 5,
  'word': 'Надежда',
  'start': 25,
  'end': 32},
 {'entity': 'LABEL_10',
  'score': 0.8543603,
  'index': 6,
  'word': 'Иванова',
  'start': 33,
  'end': 40},
 {'entity': 'LABEL_0',
  'score': 0.99496657,
  'index': 7,
  'word': 'получила',
  'start': 41,
  'end': 49},
 {'entity': 'LABEL_0',
  'score': 0.9948441,
  'index': 8,
  'word': 'статус',
  'start': 50,
  'end': 56},
 {'entity': 'LABEL_0',
  'score': 0.99499434,
  'index': 9,
  'word': 'зампред',
  'start': 57,
  'end': 64},
 {'entity': 'LABEL_0',
  

In [102]:
res = ner_pipeline(text)

In [117]:
len(ann_f)

39

In [130]:
get_entities(res, len(ann_f), int_tags)

[('департамента', 'B-ORGANIZATION'), ('ЦБ', 'B-ORGANIZATION'), ('РФ', 'U-COUNTRY'), ('Надежда', 'B-PERSON'), ('Иванова', 'L-PERSON'), ('экономический', 'B-ORGANIZATION'), ('департамент', 'I-ORGANIZATION'), ('Банка', 'L-ORGANIZATION'), ('России', 'U-COUNTRY'), ('свод', 'I-ORGANIZATION'), ('##ного', 'I-ORGANIZATION'), ('экономического', 'I-ORGANIZATION'), ('департамента', 'I-ORGANIZATION'), ('Банка', 'L-ORGANIZATION'), ('России', 'U-COUNTRY'), ('Надежда', 'B-PERSON'), ('Иванова', 'L-PERSON'), ('Банка', 'B-ORGANIZATION'), ('России', 'U-COUNTRY'), ('Гос', 'B-ORGANIZATION'), ('##банка', 'L-ORGANIZATION'), ('СССР', 'L-ORGANIZATION'), ('экономический', 'I-ORGANIZATION'), ('департамент', 'L-ORGANIZATION'), ('Иванова', 'U-PERSON'), ('департамента', 'L-ORGANIZATION'), ('экономический', 'I-ORGANIZATION'), ('департамент', 'L-ORGANIZATION'), ('Банка', 'B-ORGANIZATION'), ('России', 'U-COUNTRY'), ('Эль', 'L-PERSON'), ('##вир', 'L-PERSON'), ('##ы', 'L-ORGANIZATION'), ('Наби', 'L-PERSON'), ('##уллин', 

IndexError: tuple index out of range